In [2]:
# Import Libraries
%matplotlib notebook
import numpy as np, matplotlib.pyplot as plt, datetime, tensorflow as tf, os, datetime
import tensorflow_probability as tfp

tfd = tfp.distributions

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical, plot_model
from keras.callbacks import ModelCheckpoint#, TensorBoard                                                                      
from keras import backend as K
#from tensorboard.plugins.hparams import api as hp                                                                             
#from keras.utils.generic_utils import get_custom_objects                                                                      

import gc
gc.enable()

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
toy_model = np.load('toy_models.npz')
nx, ny, ntrain = toy_model['training_data'].shape
training_data = toy_model['training_data'].T
labels = toy_model['labels']
outputdir = "./"

HP_EPOCH = 15
HP_BATCH_SIZE = 5

In [ ]:
# Data Preprocessing                                                                                                                  
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 28, 28, 1) / 255.0
X_test = X_test.reshape(10000, 28, 28, 1) / 255.0
X_train = training_data[0:8000,:,:].reshape(8000,nx,ny,1)
y_train = labels[0:8000]*100
X_test = training_data[8000:,:,:].reshape(2000,nx,ny,1)
y_test = labels[8000:]*100

print("training", X_train.shape)
print("validation", X_test.shape)

In [ ]:
# Custom Loss Functions                                                                                                               
kl_divergence = tf.keras.losses.KLDivergence()

neg_log_likelihood = lambda y_true, y_pred: -tf.reduce_mean(input_tensor=y_pred.log_prob(y_true))

def Mean_Squared_over_true_Error(y_true, y_pred):
    # Create a custom loss function that divides the difference by the true                                                           

    y_true = K.cast(y_true, y_pred.dtype) #Casts a tensor to a different dtype and returns it.                                        
    diff_ratio = K.square((y_pred - y_true)/K.clip(K.abs(y_true),K.epsilon(),None))

    loss = K.mean(diff_ratio, axis=-1)

    # Return a tensor                                                                                                                 
    return loss

def elbo(y_true, y_pred):
    kl_weight = 1
    neg_log_likelihood = -tf.reduce_mean(input_tensor=y_pred.log_prob(y_true))
    kl_divergence = tf.keras.losses.KLDivergence()

    elbo_loss = -tf.math.reduce_mean(-kl_weight * kl_divergence(y_true, y_pred.mean()) - neg_log_likelihood)
    # Return a tensor                                                                                                                 
    return elbo_loss

In [ ]:
# Adapt TensorFlow runs to log hyperparameters and metrics
def train_test_model():
    # Build model                                                                                                                                                                                                               
    model = Sequential()
    # Add layers                                                                                                                                                                                                                
    model.add(Conv2D(16, kernel_size=3, activation="relu", input_shape=(nx, ny, 1)))                                                                                                                                            
    #model.add(BatchNormalization())                                                                                                                                                                                            
    #model.add(MaxPooling2D(pool_size=(2, 2)))                                                                                                                                                                                  
    model.add(Conv2D(8, kernel_size=3, activation="relu"))                                                                                                                                                                      
    #model.add(BatchNormalization())                                                                                                                                                                                            
    #model.add(MaxPooling2D(pool_size=(2, 2)))                                                                                                                                                                                  
    model.add(Flatten())                                                                                                                                                                                                        
    model.add(tfp.layers.DenseFlipout(1, activation="linear"))                                                                                                                                                                  
    #model.add(Dense(1,activation="linear"))                                                                                                                                                                                    
    model.add(tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t, scale=1)))                                                                                                                                              
                                                                                                                                                                                                                                
    model.compile(optimizer= "adam", loss = "mse")#"mape")                                                                                                                                                                      
                                                                                                                                                                                                                                
    # Visualize Model                                                                                                                                                                                                           
    print(model.summary())
    plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    
    model.fit(X_train, y_train, validation_data=(X_test, y_test),                                                                                                                                                               
                epochs=HP_EPOCH, batch_size=HP_BATCH_SIZE,                                                                                                                                                                      
                #callbacks = callbacks_list)                                                                                                                                                                                    
               )       
    
    loss = model.evaluate(X_test, y_test)                                                                                                                                                                                       
    return model

In [ ]:
# GENERAL Model

def train_test_model():
    # Making a model.                                                                                                                                                                   
    model = Sequential()

    # Added Layers.                                                                                                                                                                     
    model.add(Conv2D(16, kernel_size=3, activation="relu", input_shape=(nx, ny, 1), padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dense(8, activation = 'relu'))
    model.add(Conv2D(8, kernel_size = 3, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())

    model.add(Dense(4, activation = 'relu'))
    model.add(Conv2D(4, kernel_size = 3, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())

    model.add(Dense(2, activation = 'relu'))
    model.add(Conv2D(2, kernel_size = 3, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())

    model.add(Dense(4, activation = 'relu'))
    model.add(Conv2D(4, kernel_size = 3, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())

    model.add(Dense(2, activation = 'relu'))
    model.add(Conv2D(2, kernel_size = 3, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())

    model.add(Dense(1, activation = 'relu'))
    model.add(Conv2D(1, kernel_size = 3, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())

    #model.add(tfp.layers.Convolution2DFlipout(1, kernel_size=3, activation = 'relu', padding = 'same'))
    model.add(Conv2D(1, kernel_size = 3, activation = 'relu', padding = 'same'))                                                                                                       
    model.add(BatchNormalization())
    model.add(Flatten())

    #model.add(Dense(1,activation="linear"))                                                                                                                                            
    #model.add(tfp.layers.DenseFlipout(1, activation="linear"))
    #model.add(tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t, scale=1)))                                                                                                     

    model.compile(optimizer= "adam", loss = "mape")#neg_log_likelihood)#"mse")#"mape")                                                                                                  

    # Visualize Model                                                                                                                                                                  \
                                                                                                                                                                                        
    print(model.summary())
    model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=HP_EPOCH, batch_size=HP_BATCH_SIZE,
                   )

    loss = model.evaluate(X_test, y_test)
    return model

In [ ]:
model = train_test_model()

In [ ]:
"""                                                                                                                                                                                     
# Convert to true tau units                                                                                                                                                             
true_tau = low_z_tau + h_2 * result["truth"][:,n]/factor                                                                                                                                
predicted_tau = low_z_tau + h_2 * result["prediction"][:,n]/factor                                                                                                                      
"""

In [ ]:
# make predictions                                                                                                                                                                      
predictions = []
for i in range(10):
    y_p = model.predict(X_test).squeeze()#predict(X_test, batch_size=test_labels.shape[0])                                                                                              
    predictions.append(y_p) # (500, 100, 1) = (# of masks, # of datasets, # of classes)                                                                                                 
predictions = np.mean(np.array(predictions), axis=0)
print(X_test.shape)
print(predictions.shape)
print(y_test.shape)

In [ ]:
plt.plot(y_test, predictions, '.')
plt.plot(y_test,y_test, "r--")

## One Layer
Goal: (1) Turn the output layer from deterministic to probabilistic **Dense(1,activation="linear") --> DenseFlipout(1, activation="linear")**.  (2) Make one-to-one prediction plots.  (3) Show that the preditioned values form a distibution that behaves apprrroximately Normal.  


### Trail 1
Loss: MAPE  

### Trial 2
Loss: MAE  

### Trial 3
Loss: NLL  
*NOTE*: Need to add the LambdaDistribution Layer because when calculating the loss it needs to calculate the negative log of a distribution.  

### Trial 4
Loss: KL Divergence  

### SUMMARY

In [ ]:
# GENEERAL Model

def train_test_model():
    # Making a model.                                                                                                                                                                   
    model = Sequential()

    # Added Layers.                                                                                                                                                                     
    model.add(Conv2D(16, kernel_size=3, activation="relu", input_shape=(nx, ny, 1), padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dense(8, activation = 'relu'))
    model.add(Conv2D(8, kernel_size = 3, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())

    model.add(Dense(4, activation = 'relu'))
    model.add(Conv2D(4, kernel_size = 3, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())

    model.add(Dense(2, activation = 'relu'))
    model.add(Conv2D(2, kernel_size = 3, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())

    model.add(Dense(4, activation = 'relu'))
    model.add(Conv2D(4, kernel_size = 3, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())

    model.add(Dense(2, activation = 'relu'))
    model.add(Conv2D(2, kernel_size = 3, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())

    model.add(Dense(1, activation = 'relu'))
    model.add(Conv2D(1, kernel_size = 3, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())

    #model.add(tfp.layers.Convolution2DFlipout(1, kernel_size=3, activation = 'relu', padding = 'same'))
    model.add(Conv2D(1, kernel_size = 3, activation = 'relu', padding = 'same'))                                                                                                       
    model.add(BatchNormalization())
    model.add(Flatten())

    #model.add(Dense(1,activation="linear"))                                                                                                                                            
    model.add(tfp.layers.DenseFlipout(1, activation="linear"))
    #model.add(tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t, scale=1)))                                                                                                     

    model.compile(optimizer= "adam", loss = "mape")#neg_log_likelihood)#"mse")#"mape")                                                                                                  

    # Visualize Model                                                                                                                                                                  \
                                                                                                                                                                                        
    print(model.summary())
    model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=HP_EPOCH, batch_size=HP_BATCH_SIZE,
                   )

    loss = model.evaluate(X_test, y_test)
    return model

model = train_test_model()

# make predictions                                                                                                                                                                      
predictions = []
for i in range(10):
    y_p = model.predict(X_test).squeeze()#predict(X_test, batch_size=test_labels.shape[0])                                                                                              
    predictions.append(y_p) # (500, 100, 1) = (# of masks, # of datasets, # of classes)                                                                                                 
predictions = np.mean(np.array(predictions), axis=0)
print(X_test.shape)
print(predictions.shape)
print(y_test.shape)

# Plot one-to-one predictions
plt.figure()
plt.title("")
plt.xlabel("Tue Tau")
plt.ylabel("Predicted Tau")
plt.plot(y_test, predictions, '.')
plt.plot(y_test,y_test, "r--")